In [ ]:
# Importing libraries using import keyword.
import math
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import pylab
from scipy import signal
import pywt
import pywt.data
import glob
import os
from pathlib import Path
np.set_printoptions(threshold=sys.maxsize)
import tensorflow as tf
#from biosppy.signals import ecg
from scipy.signal import butter, filtfilt
from scipy import fftpack
import scipy.signal as signal
np.set_printoptions(suppress=True)
from scipy.signal import butter, lfilter
from scipy.signal import savgol_filter

#Global variables

denoised_signal = np.array([])
ecg_peaks_array=np.array([])
ecg_peaks_indexes=np.array([])
duplicate_matching_array = []
global_dataframe = pd.DataFrame(columns=['channel','order'])
blank_augmented_array = np.zeros((5000, 12))
output_counter=0


In [ ]:
def Calculate_final_peaks(ecg_signal,Lead_direction,sampling_rate):

    peaks_result = np.array([])

    if Lead_direction == "positive" or Lead_direction == "biphasic":

      out = ecg.ecg(signal=ecg_signal, sampling_rate=sampling_rate, show=False)
      peaks_result = out['rpeaks']

    else:

      out = ecg.ecg(signal= -(ecg_signal), sampling_rate=sampling_rate, show=False)
      peaks_result = out['rpeaks']

    return peaks_result

In [ ]:
def find_Rpeaks(ecg_signal,ecg_direction,lead,highcut=4,lowcut=0.1):

  #global filtered_ecg
  #global r_peaks
  #global row


  #checking ecg direction
  copy_ecg_signal = ecg_signal

  if ecg_direction == 'negative':
    copy_ecg_signal = - (copy_ecg_signal)
  else:
    copy_ecg_signal = ecg_signal

  b, a = butter(highcut, lowcut, btype='high', analog=False)  #to be define highcut, lowcut

  # Apply the filter to the ECG signal
  filtered_ecg = filtfilt(b, a, copy_ecg_signal)

  # Calculate the 97th percentile value
  percentile_value = np.percentile(filtered_ecg, 98)

  # Filter the array to get the top 20% points
  top_20_percent = filtered_ecg[filtered_ecg >= percentile_value]

  top_20_percent_indices = np.where(filtered_ecg >= percentile_value)[0]
  #print("before avg: ", top_20_percent_indices)
  # calculate avg value of indices
  avg_indices=0
  for t in top_20_percent_indices:
    avg_indices+=filtered_ecg[t]

  avg_indices = avg_indices / len(top_20_percent_indices)
  print(avg_indices)

  filtered_top_20_percent_indices = []
  for t in top_20_percent_indices:
    if filtered_ecg[t] >= avg_indices:
      filtered_top_20_percent_indices.append(t)

  top_20_percent_indices= filtered_top_20_percent_indices
  #print("after avg: ", top_20_percent_indices)
  # Calculate the minimum distance between percentiled points to filterout extra elements
  min_distance = 35

  # Initialize an empty list to store the selected indices
  peaks = []

  # Add the first point to the selected_indices
  peaks.append(top_20_percent_indices[0])
  check_point=top_20_percent_indices[0]

  # Iterate through the indices array, starting from the second index (index 1)
  for i in range(1, len(top_20_percent_indices)):

      if (top_20_percent_indices[i] - check_point) >= min_distance:
        peaks.append(top_20_percent_indices[i])
        check_point = top_20_percent_indices[i]

  #Peaks adjustment and plot

  refined_peaks=np.array([])
  if ecg_direction == 'negative':
    for n in peaks:

      i=n
      while True:
        if n < (len(ecg_signal)-4):
            if ecg_signal[n] <= ecg_signal[n+1] and ecg_signal[n] <= ecg_signal[n-1]:
              refined_peaks=np.append(refined_peaks,n)
              n=i
              break
            else:
                if ecg_signal[n-1] < ecg_signal[n]:
                  n=n-1
                else:
                  n=n+1
        else:
            break
  else:
      for n in peaks:
        i=n
        while True:
            if n < (len(ecg_signal)-4):
                if ecg_signal[n] >= ecg_signal[n+1] and ecg_signal[n] >= ecg_signal[n-1]:
                    refined_peaks=np.append(refined_peaks,n)
                    n=i
                    break
                else:
                    if ecg_signal[n-1] > ecg_signal[n]:
                        n=n-1
                    else:
                        n=n+1
            else:
                break


  r_peaks=refined_peaks

  #plt.figure(figsize=(18, 6))
  #plt.plot(ecg_signal)
  #plt.scatter(r_peaks.astype(int),ecg_signal[r_peaks.astype(int)],label='R-Peaks',color='red', s=60)

  return r_peaks

In [ ]:
def signal_peaks_arrangement(filtered_peaks_result,lead_no,file_name,dest_dir,current_duplicate,Is_Last):

  global blank_augmented_array
  global ecg_peaks_array
  global denoised_signal
  global ecg_peaks_indexes
  global duplicate_matching_array
  global global_dataframe

  temp_array=np.array([0])
  final_augmented_ecg=np.array([])


##### Data frame arrangement

  filtered_peaks_result = np.round(filtered_peaks_result,6)
  df = pd.DataFrame(columns=['Start_Index',"End_Index", 'Peak_Value'])

  row=0
  for i in filtered_peaks_result:

    if row==0:
      df.loc[row, 'Start_Index'] = 0
      df.loc[row, 'End_Index'] = int(i)
    else:
      df.loc[row, 'Start_Index'] = int(df.iloc[(row-1),1])
      df.loc[row, 'End_Index'] = int(i)
    row+=1

  df.loc[df.shape[0]-1, 'End_Index']=int(len(denoised_signal))
  ####print(df,"\n\n\n")



#### Shuffling data frame rows (except first and last)

  # Generate a permutation of indices for the middle rows
  if df.shape[0]>1:

    df_rows = np.arange(df.shape[0])
    ##print("df shape ",df_rows)

    middle_rows = df_rows[1:-1]
    ##print("Middle rows ",middle_rows)

    new_order = np.random.permutation(middle_rows)
    ##print("new_order ",new_order)
    ##print("SUM",np.sum(middle_rows == new_order))
    #print(df)

    if current_duplicate == 0:
      duplicate_matching_array.append(new_order)

    if len(middle_rows) > 1:
      matching_percentage = np.sum(middle_rows == new_order) / len(middle_rows) * 100
      ##print("######## matching percentage ############",matching_percentage)
      while True:
        #if matching_percentage >= 5:
        if matching_percentage >= 40:  #if matching percentage between actual middle rows and re-order middle rows is greater than 40% then reshuffle again
          new_order = np.random.permutation(middle_rows)
          matching_percentage = np.sum(middle_rows == new_order) / len(middle_rows) * 100
        else:
          #print(global_dataframe)
          global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
          print("################### match percentage is ok #############")
          break

    no_of_check = 0
    # Iterate over rows and access the 'order' column
    for index, row in global_dataframe.iloc[:-1].iterrows():  #iterate over the complete dataframe except last row because it has the current channel order data
      if row['channel'] == lead_no:
        matching_percentage = np.sum(row['order'] == new_order) / len(middle_rows) * 100
        while True:
          if matching_percentage > 50 and no_of_check < 4:
              ##print("I am in while true for global matching")
              #delete the last row from global dataframe
              global_dataframe = global_dataframe.drop(global_dataframe.index[-1])
              #generate a new order
              new_order = np.random.permutation(middle_rows)
              matching_percentage = np.sum(row['order'] == new_order) / len(middle_rows) * 100
              #add this new order to the global dataframe
              global_dataframe = global_dataframe.append({'channel': lead_no, 'order': new_order}, ignore_index=True)
              #increment no of check, in case there is no possibilty to generate unique order then leave it
              no_of_check+=1
          else:
            no_of_check = 0
            break

    ##print("######## new order with glboal matching percentage ############",new_order,"---- %",matching_percentage)


    new_order = np.insert(new_order, 0, 0)
    ##print("new_order after inserting 0: ",new_order)
    temp_array = np.append(new_order, df.shape[0]-1)
    ##print("temp_array after inserting last df item: ",temp_array)

    for j in temp_array:
        #print("value of j: :::::::::::::", j)
        j=int(j)
        final_augmented_ecg= np.append(final_augmented_ecg,denoised_signal[df.loc[j,'Start_Index']:df.loc[j,'End_Index']])
  else:
    final_augmented_ecg= np.append(final_augmented_ecg,denoised_signal[df.loc[0,'Start_Index']:df.loc[0,'End_Index']])


  ##### Inserting rearranged peaks into augmented array

  #print("final_augmented_ecg length ", file_name, current_duplicate ,len(final_augmented_ecg))

  #blank_augmented_array[:len(final_augmented_ecg), lead_no] = final_augmented_ecg
  ##print(len(final_augmented_ecg))

  if final_augmented_ecg.shape[0] == blank_augmented_array.shape[0]:
    blank_augmented_array[:, lead_no] = final_augmented_ecg
  else:
    print("Error: Could not assign final_augmented_ecg to blank_augmented_array due to shape mismatch.")

  #blank_augmented_array[:, lead_no] = final_augmented_ecg
  ###graph_display(final_augmented_ecg)

  #if file_name == '00138' and lead_no == 9:
 #   test()

  denoised_signal = np.array([])
  ecg_peaks_array=np.array([])
  ecg_peaks_indexes=np.array([])

 # print("after: leng of final_augmented_ecg: ", len(final_augmented_ecg))
 # print("after: leng of denoised signal: ", len(denoised_signal))

  if lead_no == 2:
    # Save the numpy array as .npy file
    np.save(str(dest_dir)+str(file_name)+"_hr_"+str(current_duplicate)+".npy", blank_augmented_array)
    #Flush array
    blank_augmented_array[:] = 0

  if Is_Last == 1:
    # Remove all rows from the original DataFrame except the header row
    global_dataframe.drop(global_dataframe.index[1:], inplace=True)
    print("############################################################################ I flush the global dataframe")

  return final_augmented_ecg

In [ ]:
def find_signal_direction(ecg_signal):

  # Find the index of the maximum amplitude in the ecg signal
  max_index = np.argmax(np.abs(ecg_signal))

  # Determine the sign of the maximum amplitude
  max_amplitude = ecg_signal[max_index]

  if max_amplitude >= 0:
      peak_side = "positive"
  elif max_amplitude < 0:
      peak_side = "negative"
  else:
      peak_side = "biphasic"

  # Print the result
  ###print(f"The peak in the segment is on the {peak_side} side.")

  return peak_side

In [ ]:
###################################################################################################### FINAL VERSION #################################################################################

def peaks_finder(signal, wavelet, denoising_level, Lead_no, File_name, Augmented_Save_dir, current_duplicate, Is_Last, sampling_rate, PositivePeaksSearch_threshold=0.0, NegativePeaksSearch_threshold=-0.0):

    global ecg_peaks_array
    global denoised_signal
    global ecg_peaks_indexes
    global output_counter

    np.set_printoptions(suppress=True)
    attempt=0
    myarray=np.array([])
    myarray_index=np.array([])

    def madev(d, axis=None):
        """ Mean absolute deviation of a signal """
        return np.mean(np.absolute(d - np.mean(d, axis)), axis)

    def wavelet_denoising(x, wavelet, level):
        coeff = pywt.wavedec(x, wavelet, mode="per")
        sigma = (1 / 0.6745) * madev(coeff[-level])
        uthresh = sigma * np.sqrt(2 * np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
        return pywt.waverec(coeff, wavelet, mode='per')

    # wavelist=['bior1.3','bior2.8','bior3.1','bior3.9','db2','db8','rbio1.5','sym6','sym8']

    filtered = wavelet_denoising(signal, wavelet, denoising_level)

    n=filtered
    now=n[0]

    slope_mean = np.mean(np.diff(n))
    ##print("slope mean: ",slope_mean)

    Pos_threshold_result=n[n > PositivePeaksSearch_threshold]  #0.1

    if len(Pos_threshold_result) > 1:

      for i in range(1, len(n)):

          if n[i] >= now:
            now=n[i]
            myarray=np.append(myarray,n[i])
            myarray_index = np.append(myarray_index,i)

          elif n[i] < now and attempt >=2:

            if len(myarray) >= 3:

              #if has_generally_increasing_trend(myarray,trend_threshold) == True: #  or has_generally_increasing_trend(myarray,trend_threshold) == False:   ## value for trend_threshold = 0.5
                ecg_peaks_array = np.append(ecg_peaks_array,np.max(myarray,keepdims=True))
                ecg_peaks_indexes = np.append(ecg_peaks_indexes, myarray_index[myarray.argmax()])

            myarray=np.array([])
            myarray_index = np.array([])
            attempt=0
            now=n[i]

          elif n[i] < now:
            attempt+=1
            myarray=np.append(myarray,n[i])
            myarray_index = np.append(myarray_index,i)

      attempt=0
      Neg_threshold_result = n[n < NegativePeaksSearch_threshold] #-0.3

      if len(Neg_threshold_result) > 4:

        myarray=np.array([])
        myarray_index=np.array([])

        for i in range(1, len(n)):

          if n[i] < 0 and len(myarray) < 50:

              myarray = np.append(myarray, n[i])
              myarray_index=np.append(myarray_index,i)

          elif n[i] >= 0 and len(myarray) >= 3:

              ecg_peaks_array=np.append(ecg_peaks_array,np.min(myarray,keepdims=True))
              ecg_peaks_indexes = np.append(ecg_peaks_indexes,myarray_index[myarray.argmin()])
              #print("length",len(myarray), "at index ", i)
              myarray=np.array([])
              myarray_index=np.array([])

          elif n[i] >= 0 and len(myarray) < 3:

            myarray=np.array([])
            myarray_index=np.array([])

          elif n[i] < 0 and len(myarray) >= 50:

            ecg_peaks_array=np.append(ecg_peaks_array,np.min(myarray,keepdims=True))
            ecg_peaks_indexes = np.append(ecg_peaks_indexes,myarray_index[myarray.argmin()])
            #print("length NEW ",len(myarray), "at index ", i)
            myarray=np.array([])
            myarray_index=np.array([])

    denoised_signal=filtered
    myarray=np.array([])
    myarray_index=np.array([])

    #plt.figure(figsize=(10, 4))

    #plt.bar(ecg_peaks_indexes, ecg_peaks_array,color='0.5')
    #plt.scatter(ecg_peaks_indexes, ecg_peaks_array,color='0.5', s=60)
    #plt.axis("off")
   # plt.savefig('bar_chart.png', bbox_inches='tight')

    result_direction = find_signal_direction(signal)
    ###### Calling supportive functions
    #result_final_peaks=Calculate_final_peaks(denoised_signal,result_direction,sampling_rate)

    result_final_peaks=find_Rpeaks(denoised_signal,result_direction,Lead_no)

    result_peaks_arrangement = signal_peaks_arrangement(result_final_peaks,Lead_no, File_name, Augmented_Save_dir, current_duplicate,Is_Last)

    #output_counter+=1
    print(str(File_name) + " --- "+ " augmentation level: " + str(current_duplicate) + " done...")


In [ ]:
def graph_display(final_augmented_ecg):

  ###################################################################################################### FINAL VERSION #################################################################################
  import matplotlib.pyplot as plt

  #print(denoised_signal)
 ### plt.figure(figsize=(15, 6))
  # Sample data
  x = final_augmented_ecg

  #plt.plot(denoised_signal, linestyle='-', color='red', label='Main Plot')

  # Create the main plot
  ###plt.plot(x, linestyle='-', color='blue', label='Main Plot')

  # Add labels and legend
  ###plt.xlabel('X-axis')
  ###plt.ylabel('Y-axis')
  ###plt.legend()

  # Show the plot
  ###plt.show()
  ###plt.close('all')

  #0-97
  #152-200

In [ ]:
###################################################################################################### FINAL VERSION #################################################################################
def test():
    import matplotlib.pyplot as plt

    plt.figure(figsize=(15, 6))
    # Sample data
    x = denoised_signal

    #print(len(ecg_peaks_indexes))
    #print(len(ecg_peaks_array))

    #print(np.array2string(ecg_peaks_array, separator=', '))


   # print(ecg_peaks_array)
    #print(ecg_peaks_indexes)

    y = ecg_peaks_indexes

    # Highlighted points
    highlighted_indices = ecg_peaks_array  # Indices of points to highlight

    # Create the main plot
    plt.plot(x, linestyle='-', color='blue', label='Main Plot')

    # Highlight specific points
    #highlighted_x = [x[i] for i in highlighted_indices]
    #highlighted_y = [y[i] for i in highlighted_indices]

    #plt.scatter(y, highlighted_indices, color='red', s=100, label='Highlighted Points')
    plt.scatter(y, highlighted_indices, color='red', s=100, label='Highlighted Points')

    # Add labels and legend
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.legend()

    # Show the plot
    plt.show()

    #0-97
    #152-200

In [ ]:
def main():


    Source_dir="/path_to/New directory/src/"
    Dest_dir="/path_to/With_Data_Aug/New directory/dest/"
    existing_file_name=""

    Req_duplicates = 2
    Leads_in_Signal = 12

    for file in glob.iglob(Source_dir + '**/*.npy', recursive=True):

        x = np.load(file) #load the selected file data array into variable x

        #extracting file name i.e., to save the augmented sample with similar name
        existing_file_name = Path(file).stem
        #existing_file_name = existing_file_name[:5]

        for rd in range (0, Req_duplicates, 1):  # Will run equal to the required duplicate signal times, to generate the multiple augmented signals

          for Lead in range (0, Leads_in_Signal, 1):  #initiating 12 times loop on variable x to extract lead/column wise data (which then passed to Host_Guest_ECG_Augmentation_12_Leads method)

              if rd == (Req_duplicates-1) and Lead == (Leads_in_Signal-1): finish = 1
              else: finish = 0

              n=x[Lead]
              #n=np.array(n).flatten()
              n=x[:,Lead]
              #out = ecg.ecg(signal=n[:3000], sampling_rate=250, show=False)
              ##print(file)
              peaks_finder(signal=n[:5000], wavelet='bior3.1', denoising_level=1, Lead_no=Lead, File_name=existing_file_name, Augmented_Save_dir=Dest_dir, current_duplicate=rd, Is_Last = finish, sampling_rate=500)

main()

0.05906901290026814
################### match percentage is ok #############
JS10681 ---  augmentation level: 0 done...
0.04550415666130398
################### match percentage is ok #############
JS10681 ---  augmentation level: 0 done...
0.0660335223020865
################### match percentage is ok #############
JS10681 ---  augmentation level: 0 done...


/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))


0.07857311521107806
################### match percentage is ok #############
JS10681 ---  augmentation level: 0 done...
0.0945895182829206
################### match percentage is ok #############
JS10681 ---  augmentation level: 0 done...
0.08228098968320252
################### match percentage is ok #############
JS10681 ---  augmentation level: 0 done...


/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))


0.03861513401223697
################### match percentage is ok #############
JS10681 ---  augmentation level: 0 done...
0.04090486849324804
################### match percentage is ok #############
JS10681 ---  augmentation level: 0 done...
0.03089157119996537
################### match percentage is ok #############
Error: Could not assign final_augmented_ecg to blank_augmented_array due to shape mismatch.
JS10681 ---  augmentation level: 0 done...


...

/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))


0.1857117320995947
################### match percentage is ok #############
JS10687 ---  augmentation level: 1 done...
0.397368262609786
################### match percentage is ok #############
JS10687 ---  augmentation level: 1 done...
0.261475589492573
################### match percentage is ok #############
JS10687 ---  augmentation level: 1 done...


/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))


0.22772516281512103
################### match percentage is ok #############
JS10687 ---  augmentation level: 1 done...
0.13097072831860018
################### match percentage is ok #############
JS10687 ---  augmentation level: 1 done...
0.14775086045836644
################### match percentage is ok #############
############################################################################ I flush the global dataframe
JS10687 ---  augmentation level: 1 done...


/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
/tmp/ipykernel_202021/1464871827.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  global_dataframe = global_dataframe.append(pd.DataFrame({'channel': lead_no, 'order': new_order}))
